In [1]:
import os
import sys
from PIL import Image

import numpy as np

import keras

from keras_frcnn import models


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_config = models.load_model_config_template()
model_config.num_classes = 7
model_config.input_shape = (800, 800, 3)

model = models.FRCNN(model_config)
model.output_shape

[(None, 300, 4), (None, 300), (None, 300)]

In [3]:
print(model.get_layer('P3').name, model.get_layer('P3').output_shape)
print(model.get_layer('P4').name, model.get_layer('P4').output_shape)
print(model.get_layer('P5').name, model.get_layer('P5').output_shape)
print(model.get_layer('P6').name, model.get_layer('P6').output_shape)
print(model.get_layer('P7').name, model.get_layer('P7').output_shape)
print(model.layers[-1].name, model.layers[-1].output_shape)

P3 (None, 97, 97, 256)
P4 (None, 48, 48, 256)
P5 (None, 23, 23, 256)
P6 (None, 12, 12, 256)
P7 (None, 6, 6, 256)
boxes (None, 111798, 4)


In [5]:
print(model.get_layer('P3').output_shape[1] *   8)
print(model.get_layer('P4').output_shape[1] *  16)
print(model.get_layer('P5').output_shape[1] *  32)
print(model.get_layer('P6').output_shape[1] *  64)
print(model.get_layer('P7').output_shape[1] * 128)


1312
1296
1280
1280
1280


In [2]:
x = np.zeros((1, 800, 1333, 3))
y = model.predict(x)


In [5]:
for temp in y:
    print(temp.shape)



(1, 189144, 4)
(1, 189144, 7)
(1, 189144, 4)


In [13]:
import keras

<tf.Tensor 'input_5:0' shape=(?, ?, ?, 3) dtype=float32>

In [22]:
temp = keras.Input(shape=(None,3))
len(temp.shape)

3

In [11]:
list(range(temp.shape[2]))

[0, 1, 2, 3, 4, 5, 6]

In [7]:
import keras
temp = keras.Input(shape=(3, 7))
